# Practicioners Seminar

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns


## Task 2

### A correlation heatmap for currencies and equity markets

In [ ]:
FILE = ""
df = pd.read_csv(FILE)
return_cor = df.pct_change().corr()
sns.heatmap(return_cor, xticks=return_cor.columns.values, yticks=return_cor.columns.values)

### Interpretation:
**correlations between, index and associated currency pair**:determinants of the optimal hedging ratio

*higher positive correllations* $\rightarrow$ imply lower desired hedges (hedging by using leverage in the foreign currency ?)

*strong negative correlations* $\rightarrow$ imply hedging effectivenes and thus higher optimal Hedging ratios

*correlations near to zero* $\rightarrow$ are interpreted to be cases in which hedging is also not desirable as heding is costly but heding does not decrease risk as measured by return variance


**correlations between two indexes**: extreme values would indicate possible economic ties

**correlations between two currency pairs**: extreme values would indicate possible economic ties
The MSCI world index has predefined weights, however highly correlated economies and currencies should be hedged with respect to the exposure to each other. Are there such cases, what would be an optimal way to treat these economies? 


